# Final Notebook

### Imports

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
import random
from PIL import Image,ImageEnhance  
import os
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
import seaborn as sns

from tensorflow.keras.utils import plot_model
from sklearn.metrics import accuracy_score

#needed for transformation
import imageio
import imgaug as ia

import imgaug.augmenters as iaa
import random



# Functions

### Data augmentation

In [ ]:
#input has to be of type image

def random_rotation(image):
    rotate=iaa.Affine(rotate=(-90,90))
    rotated_image=rotate.augment_image(image)
    image_from_array = Image.fromarray(rotated_image, 'RGB')
    size_image = image_from_array.resize((32, 32))
    size_image=size_image.convert('L')
    return size_image

def random_noise(image):
    gaussian_noise=iaa.AdditiveGaussianNoise(10,20)
    noise_image=gaussian_noise.augment_image(image)
    image_from_array = Image.fromarray(noise_image, 'RGB')
    size_image = image_from_array.resize((32, 32))
    size_image=size_image.convert('L')
    return size_image





# Train Data


In [ ]:

    
# Reading the input images and putting them into a numpy array

#this is for using random transformations
available_transformations = {
    'rotate': random_rotation,
    'noise': random_noise}

num_transformations_to_apply = random.randint(1, len(available_transformations))
num_transformations = 0
transformed_image = None

data=[]
labels=[]

height = 32
width = 32
channels = 3
classes = 43
n_inputs = height * width*channels

for i in range(classes) :
    path = "INSERT_PATH".format(i)
    #print(path)
    Class=os.listdir(path)
    for a in Class:
        try:
            
            image=cv2.imread(path+a)
            image_from_array = Image.fromarray(image, 'RGB') 
            size_image = image_from_array.resize((height, width))
            
            #Converting to grayscale and applying sharpness
            size_image=size_image.convert('L')
            enh=ImageEnhance.Sharpness(size_image)
            data.append(np.array(enh.enhance(2.0)))
            labels.append(i) 
   

            #random transformation
            key = random.choice(list(available_transformations))
            transformed_image = available_transformations[key](image)
            
            #adding augmented image
            data.append(np.array(transformed_image))
            labels.append(i)
            
            
        except AttributeError:
            print(" ")
            
Cells=np.array(data)
labels=np.array(labels)

#Randomize the order of the input images
s=np.arange(Cells.shape[0])
np.random.seed(43)
np.random.shuffle(s)
Cells=Cells[s]
labels=labels[s]

n=0
while n <= 10:
    ia.imshow(data[n])
    n= n+1

# Splitting Data

In [ ]:
(X_train,X_val)=Cells[(int)(0.2*len(labels)):],Cells[:(int)(0.2*len(labels))]
X_train = X_train.astype('float32')/255 
X_val = X_val.astype('float32')/255
(y_train,y_val)=labels[(int)(0.2*len(labels)):],labels[:(int)(0.2*len(labels))]

In [ ]:
 fig = sns.distplot(y_train, kde=False, bins = 43, hist = True, hist_kws=dict(edgecolor="black", linewidth=2))
fig.set(title = "Traffic signs frequency graph",
        xlabel = "ClassId",
        ylabel = "Frequency")
X_train.shape[0]


In [ ]:
from keras.utils import to_categorical
y_train = to_categorical(y_train, 43)
y_val = to_categorical(y_val, 43)

# Visualisation of Data Set

# Test Data

In [ ]:

y_test=pd.read_csv("C:/Users/user-pc/Documents/Advanced Machine Learning/gtsrb-german-traffic-sign/Test.csv")
labels=y_test['Path'].to_numpy()
y_test=y_test['ClassId'].values

data=[]

for f in labels:
    image=cv2.imread('C:/Users/user-pc/Documents/Advanced Machine Learning/gtsrb-german-traffic-sign/test/'+f.replace('Test/', ''))
    image_from_array = Image.fromarray(image, 'RGB')
    size_image = image_from_array.resize((height, width))
    
     #Converting to grayscale and applying sharpness
    size_image=size_image.convert('L')
    enh=ImageEnhance.Sharpness(size_image)
    data.append(np.array(enh.enhance(2.0)))
    
            
  
   
X_test=np.array(data)
X_test = X_test.astype('float32')/255 

i=0
while i <= 10:
    ia.imshow(data[i])
    i=i+1


In [ ]:
fig2 = sns.distplot(y_test, kde=False, bins = 43, hist = True, hist_kws=dict(edgecolor="black", linewidth=2))
fig2.set(title = "Traffic signs frequency graph",
        xlabel = "ClassId",
        ylabel = "Frequency")

# Preprocessing

In [ ]:
#grayscale images 

X_train = np.reshape(X_train,(X_train.shape[0],32,32,1))
X_val = np.reshape(X_val,(X_val.shape[0],32,32,1))
X_test = np.reshape(X_test,(X_test.shape[0],32,32,1))



In [ ]:
#amount of training samples
X_train.shape[0]

# Model Creation

In [ ]:
model = keras.Sequential([  
    Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=X_train.shape[1:]),
    Dropout(0.4),
    MaxPooling2D(pool_size=(2,2), strides=(2,2),padding='same',),

    Conv2D(filters=64, kernel_size=(5,5), activation='relu'),
    Dropout(0.3),
    MaxPooling2D(pool_size=(2,2), strides=(2,2)),
    


    Flatten(),
    
    Dense(units=1024, activation='relu'),
    Dropout(0.2),

    Dense(units=43, activation = 'softmax')
])

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', 
              loss='categorical_crossentropy',
              metrics=['accuracy'])


# Training

In [ ]:
vis=model.fit(X_train,y_train,epochs=30,validation_data=(X_val, y_val))

In [ ]:
plt.figure(0)
plt.plot(vis.history['accuracy'], label='training accuracy')
plt.plot(vis.history['val_accuracy'], label='val accuracy')
plt.title('Accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()

plt.figure(1)
plt.plot(vis.history['loss'], label='training loss')
plt.plot(vis.history['val_loss'], label='val loss')
plt.title('Loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()

# Testing

In [ ]:
pred = model.predict_classes(X_test)

In [ ]:
accuracy_score(y_test, pred)

# Testing on Random images

In [ ]:
images = []
for filename in os.listdir("C:/Users/user-pc/Downloads/Images-20200527T224630Z-001/Images"):
    img = cv2.imread(os.path.join("C:/Users/user-pc/Downloads/Images-20200527T224630Z-001/Images", filename))
    img=Image.fromarray(img, 'RGB')
    img = img.resize((32, 32))
    img=img.convert('L')
    plt.show(img)
    images.append(np.array(img))
    
new_img= np.array(images)   
new_img = new_img.astype('float32')/255


In [ ]:
new_img = np.reshape(new_img,(new_img.shape[0],32,32,1))

class_pred = [3,13,17,28,14]

In [ ]:
pred = model.predict_classes(new_img)

In [ ]:
print(pred)

In [ ]:
accuracy_score(class_pred, pred)